In [1]:
import csv
import time
import os
import subprocess
import psutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


class CustomCallback(keras.callbacks.Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()
        self.training_time = None
        self.average_memory_utilization = None
        self.average_gpu_utilization = None
        
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        self.memory_utilizations = []
        self.gpu_utilizations = []
        
    def on_train_end(self, logs={}):
        self.training_time = time.time() - self.start_time
        self.average_memory_utilization = np.mean(self.memory_utilizations)
        self.average_gpu_utilization = np.mean(self.gpu_utilizations)
        print('\nTotal training time: {} seconds'.format(self.training_time))
        print('\nAverage memory utilization during training so far: {}'.format(self.average_memory_utilization))
        print('\nAverage GPU utilization during training so far: {}'.format(self.average_gpu_utilization))

    def on_epoch_end(self, epoch, logs={}):
        memory_utilization = psutil.cpu_percent()
        gpu_utilization_process = subprocess.Popen(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader'], stdout=subprocess.PIPE)
        gpu_utilization = int(gpu_utilization_process.stdout.readline().strip().decode('utf-8').split()[0])
        self.memory_utilizations.append(memory_utilization)
        self.gpu_utilizations.append(gpu_utilization)

        
def Classify(batch_size, epochs):
    # Load the MNIST dataset
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Preprocess the data
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    # Define the model
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    callback = CustomCallback()
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[callback])

    # Evaluate the model
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print('Test accuracy:', test_acc)
    
    with open('mnist_classification.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([batch_size, epochs, callback.average_memory_utilization, callback.average_gpu_utilization, callback.training_time, test_loss, test_acc])

        
for batch in range(100, 801, 10):
    for epoch in range(10, 41, 10):
        Classify(batch, epoch)



2023-02-12 07:28:18.763502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 07:28:18.782583: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 07:28:18.782798: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 07:28:18.783833: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.